In [1]:
from glob import glob

import numpy as np

import matplotlib.pyplot as plt

import cv2 as cv

from astropy.io import fits

# EMPTY PSF CHECK

In [2]:
real_psf_dir = glob('/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/*psfstack.fits')

In [3]:
real_psf_dir

['/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386-a3001_t004_nircam_f356w-maskrnd-sub320a335r_psfstack.fits',
 '/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386-a3001_t004_nircam_f300m-maskrnd-sub320a335r_psfstack.fits',
 '/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386-a3001_t004_nircam_f410m-maskrnd-sub320a335r_psfstack.fits',
 '/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386-a3001_t004_nircam_f360m-maskrnd-sub320a335r_psfstack.fits']

In [5]:
def create_psd(psf):
    # cropped_psf = psf[
    #     int(psf.shape[0]*3/8):int(psf.shape[0]*5/8),
    #     int(psf.shape[1]*3/8):int(psf.shape[1]*5/8)
    #     ]
    psf = np.nan_to_num(psf)
    psd = np.abs(np.fft.fftshift(np.fft.fft2(psf)))**2
    psd = np.log10(psd)
    psd = psd/psd.max()
    return psd

In [6]:
def is_psf_empty(psf):
    width, height = psf.shape[0], psf.shape[1]
    psd = create_psd(psf)
    if psd[int(width/2) -1][int(height/2) -1] < 0.85:
        return True
    else:
        return False

In [7]:
for _, psfstackdir in enumerate(real_psf_dir):
   psfstack = fits.open(psfstackdir)[1].data

   print(psfstack.shape)
   x = 0

   for _, psf in enumerate(psfstack):
     if is_psf_empty(psf):
        x=x+1

   print(psfstackdir.split('/')[-1], x)

(18, 320, 320)
jw01386-a3001_t004_nircam_f356w-maskrnd-sub320a335r_psfstack.fits 0
(466, 320, 320)
jw01386-a3001_t004_nircam_f300m-maskrnd-sub320a335r_psfstack.fits 72
(18, 320, 320)
jw01386-a3001_t004_nircam_f410m-maskrnd-sub320a335r_psfstack.fits 0
(296, 320, 320)
jw01386-a3001_t004_nircam_f360m-maskrnd-sub320a335r_psfstack.fits 75


In [ ]:
psfstack = fits.open(real_psf_dir[0])[1].data

psd = create_psd(np.nan_to_num(psfstack[10]))

width, height = psd.shape[0], psd.shape[1]

psd = psd * 1 /np.max(psd)

print(psd[int(width/2)][int(height/2)], int(width/2), int(height/2), psd.shape)

psd[int(width/2)][int(height/2)] = 0

cropped_psf = psf[
        int(psf.shape[0]*3/8):int(psf.shape[0]*5/8),
        int(psf.shape[1]*3/8):int(psf.shape[1]*5/8)
        ]

plt.imshow(cropped_psf)

print(real_psf_dir[1])

In [ ]:
psfstack = fits.open(real_psf_dir[2])[1].data

count_of_psf = psfstack.shape[0]

plt.subplots(31, 15, figsize=(5, 20))
plt.subplots_adjust(bottom=10, right=8, top=15)


for idx, psf in enumerate(psfstack[0:465]):

    crop_coef = [
        1/4, 
        3/4
    ]
    
    psf_crop = psf[
        int(psf.shape[0]*crop_coef[0]):int(psf.shape[0]*crop_coef[1]),
        int(psf.shape[1]*crop_coef[0]):int(psf.shape[1]*crop_coef[1])
        ]
    plt.title(f'mean {np.round(np.nanmean(psf), 3)}', fontstyle='italic')
    plt.subplot(31, 15, idx+1)
    plt.imshow(psf_crop, aspect='auto')

In [ ]:
psfstack_npy_dir = glob(f'/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/injections/*.npy')

plt.subplots(30, 15, figsize=(20, 20))
# plt.subplots_adjust(bottom=10, right=8, top=15)


for idx in range(450):
    random_idx = np.random.randint(0, len(psfstack_npy_dir))
    psf_npy = np.load(psfstack_npy_dir[random_idx])

    plt.subplot(30, 15, idx+1)
    plt.axis('off')
    plt.imshow(psf_npy)

# INJECTION TEST

In [ ]:
injection_folder = '/data/scratch/bariskurtkaya/dataset/PSF_INJECTION/'

injections_paths = glob(f'{injection_folder}/*')

In [ ]:
path = injections_paths[2]

img = cv.imread(f'{path}', 0)

print(path, img.shape)
plt.imshow(np.arcsinh(img))